In [1]:
from datasets import load_dataset

ds_ShenLab = load_dataset("ShenLab/MentalChat16K")
ds_Amod = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Interview_Data_6K.csv:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

Synthetic_Data_10K.csv:   0%|          | 0.00/32.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [2]:
ds_ShenLab = ds_ShenLab['train']
ds_Amod = ds_Amod['train']

In [3]:
df_ShenLab = ds_ShenLab.to_pandas()
df_Amod = ds_Amod.to_pandas()

In [4]:
df_Shen = df_ShenLab.iloc[:,1:]

In [5]:
df_Amod_renamed = df_Amod.rename(columns={
    'Context': 'input',
    'Response': 'output'
})

In [6]:
import pandas as pd

In [7]:
df = pd.concat([df_Shen, df_Amod_renamed])

In [8]:
df.shape

(19596, 2)

In [9]:
df

,input,output
0,I've been struggling with my mental health for...,I understand that you've been dealing with a s...
1,I've been feeling overwhelmed with my caregivi...,"Your situation is complex, and it's important ..."
2,I've been feeling constantly anxious and unabl...,I can see that you're dealing with a great dea...
3,"My mom has Alzheimer's, and I've been her prim...",I'm sorry to hear that your siblings' demands ...
4,"I've tried setting boundaries, but it feels li...","Your concerns are valid, and it's crucial to p..."
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [12]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.1 MB/s eta 0:00:00


In [13]:
import re
import contractions

def clean_text(text):
    if pd.isna(text):
        return ""

    text = str(text)

    # Expand contractions (can't → cannot)
    text = contractions.fix(text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove chat role tags
    text = re.sub(r'\b(user|assistant|bot|therapist)\s*:', '', text, flags=re.I)

    # Standardize punctuation
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')

    # Remove repeated punctuation
    text = re.sub(r'([.!?]){2,}', r'\1', text)

    # Trim
    text = text.strip()

    return text

In [14]:
df_clean_input = df['input'].apply(clean_text)
df_clean_output = df['output'].apply(clean_text)

In [15]:
df_clean_input

,input
0,I have been struggling with my mental health f...
1,I have been feeling overwhelmed with my caregi...
2,I have been feeling constantly anxious and una...
3,"My mom has Alzheimer's, and I have been her pr..."
4,"I have tried setting boundaries, but it feels ..."
...,...
3507,My grandson's step-mother sends him to school ...
3508,My boyfriend is in recovery from drug addictio...
3509,The birth mother attempted suicide several tim...
3510,I think adult life is making him depressed and...


In [16]:
df_clean_output

,output
0,I understand that you have been dealing with a...
1,"Your situation is complex, and it is important..."
2,I can see that you are dealing with a great de...
3,I am sorry to hear that your siblings' demands...
4,"Your concerns are valid, and it is crucial to ..."
...,...
3507,Absolutely not! It is never in a child's best ...
3508,I am sorry you have tension between you and yo...
3509,"The true answer is, ""no one can really say wit..."
3510,How do you help yourself to believe you requir...


In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")

def split_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]


In [18]:
def decompose_compound_sentences(sentence):
    doc = nlp(sentence)

    chunks = []
    current = []

    for token in doc:
        if token.dep_ == "cc" and token.text.lower() in ["and", "but"]:
            chunks.append(" ".join(current).strip())
            current = []
        else:
            current.append(token.text)

    if current:
        chunks.append(" ".join(current).strip())

    return chunks


In [19]:
def decompose_text(text):
    simple_statements = []

    for sent in split_sentences(text):
        parts = decompose_compound_sentences(sent)
        simple_statements.extend(parts)

    return simple_statements


In [20]:
df_decomposed_text = df_clean_input.apply(decompose_text)

In [21]:
print(df_decomposed_text[:])

0       [I have been struggling with my mental health ...
1       [I have been feeling overwhelmed with my careg...
2       [I have been feeling constantly anxious, unabl...
3       [My mom has Alzheimer 's ,, I have been her pr...
4       [I have tried setting boundaries ,, it feels l...
                              ...                        
3507    [My grandson 's step - mother sends him to sch...
3508    [My boyfriend is in recovery from drug addicti...
3509    [The birth mother attempted suicide several ti...
3510    [I think adult life is making him depressed, w...
3511    [I just took a job that requires me to travel ...
Name: input, Length: 19596, dtype: object


In [22]:
df = pd.DataFrame({
    'input': df_decomposed_text,
    'output': df_clean_output
})

In [23]:
df

,input,output
0,[I have been struggling with my mental health ...,I understand that you have been dealing with a...
1,[I have been feeling overwhelmed with my careg...,"Your situation is complex, and it is important..."
2,"[I have been feeling constantly anxious, unabl...",I can see that you are dealing with a great de...
3,"[My mom has Alzheimer 's ,, I have been her pr...",I am sorry to hear that your siblings' demands...
4,"[I have tried setting boundaries ,, it feels l...","Your concerns are valid, and it is crucial to ..."
...,...,...
3507,[My grandson 's step - mother sends him to sch...,Absolutely not! It is never in a child's best ...
3508,[My boyfriend is in recovery from drug addicti...,I am sorry you have tension between you and yo...
3509,[The birth mother attempted suicide several ti...,"The true answer is, ""no one can really say wit..."
3510,"[I think adult life is making him depressed, w...",How do you help yourself to believe you requir...


In [24]:
rag_rows = []

for idx, row in df.iterrows():
    user_atomic_statements = row['input']  # this is a list
    assistant_response = row['output']

    for atom in user_atomic_statements:
        rag_rows.append({
            "text": atom,
            "source": "mental_chat",
            "type": "user_expression",
            "parent_text": " | ".join(user_atomic_statements),  # optional, original context
            "response": assistant_response
        })


In [25]:
rag_rows_flat = []

for idx, row in df.iterrows():
    atomic_statements = row['input']  # list of atomic statements
    # You can ignore the response for now
    for atom in atomic_statements:
        rag_rows_flat.append({
            "text": atom,
            "source": "mental_chat",
            "type": "user_expression",
            "parent_text": " | ".join(atomic_statements)
        })

rag_rows_flat[0]

{'text': 'I have been struggling with my mental health for a while now ,',
 'source': 'mental_chat',
 'type': 'user_expression',
 'parent_text': 'I have been struggling with my mental health for a while now , | I can not seem to find a way to cope with it . | I have tried visualization , positive thinking , | even medication , | nothing seems to work . | I have been feeling lost | helpless , | I do not know what to do next . | My mind is a whirlwind of thoughts | emotions , | I can not seem to make sense of it all . | I feel like I am drowning in a sea of confusion , | I can not seem to find my way out .'}

In [26]:
rag_df = pd.DataFrame(rag_rows_flat)

In [27]:
rag_df

,text,source,type,parent_text
0,I have been struggling with my mental health f...,mental_chat,user_expression,I have been struggling with my mental health f...
1,I can not seem to find a way to cope with it .,mental_chat,user_expression,I have been struggling with my mental health f...
2,"I have tried visualization , positive thinking ,",mental_chat,user_expression,I have been struggling with my mental health f...
3,"even medication ,",mental_chat,user_expression,I have been struggling with my mental health f...
4,nothing seems to work .,mental_chat,user_expression,I have been struggling with my mental health f...
...,...,...,...,...
192022,I really need this job .,mental_chat,user_expression,I just took a job that requires me to travel f...
192023,"People keep telling me I have "" anxiety """,mental_chat,user_expression,I just took a job that requires me to travel f...
192024,I am terrified of having an anxiety attack on ...,mental_chat,user_expression,I just took a job that requires me to travel f...
192025,This is all new to me .,mental_chat,user_expression,I just took a job that requires me to travel f...


In [28]:
five_ps_chunks = [
    {
        "text": "List the Presenting Problems the client is facing.",
        "source": "worksheet",
        "technique": "5Ps-case-formulation",
        "step": 1
    },
    {
        "text": "Identify Predisposing Factors: what has happened in the past that makes the problem more likely?",
        "source": "worksheet",
        "technique": "5Ps-case-formulation",
        "step": 2
    },
    {
        "text": "Identify Precipitating Factors: probable triggers for the current problem.",
        "source": "worksheet",
        "technique": "5Ps-case-formulation",
        "step": 3
    },
    {
        "text": "Identify Perpetuating Factors: what keeps the problem stuck, what maintains the behavior?",
        "source": "worksheet",
        "technique": "5Ps-case-formulation",
        "step": 4
    },
    {
        "text": "Identify Protective Factors: what are the positives to be developed or supported?",
        "source": "worksheet",
        "technique": "5Ps-case-formulation",
        "step": 5
    }
]


In [29]:
seven_column_chunks = [
    {
        "text": "Describe the Situation / Trigger that led to the emotional response.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 1
    },
    {
        "text": "Identify your Feelings / Emotions and rate them from 0–100%.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 2
    },
    {
        "text": "Notice your Body Sensations related to the emotion.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 3
    },
    {
        "text": "Write down Unhelpful Thoughts / Images that came to mind.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 4
    },
    {
        "text": "Record Facts that support the unhelpful thought.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 5
    },
    {
        "text": "Record Facts that provide evidence against the unhelpful thought.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 6
    },
    {
        "text": "Generate Alternative, more realistic and balanced perspectives.",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 7
    },
    {
        "text": "Reflect on your Outcome: what are you feeling now (0–100%), what could you do differently, and what will be most helpful for the situation?",
        "source": "worksheet",
        "technique": "7-column-thought-record",
        "step": 8
    }
]


In [30]:
import pandas as pd

worksheet_chunks = five_ps_chunks + seven_column_chunks
worksheet_df = pd.DataFrame(worksheet_chunks)

# Add chunk_id
worksheet_df['chunk_id'] = range(len(worksheet_df))

worksheet_df


,text,source,technique,step,chunk_id
0,List the Presenting Problems the client is fac...,worksheet,5Ps-case-formulation,1,0
1,Identify Predisposing Factors: what has happen...,worksheet,5Ps-case-formulation,2,1
2,Identify Precipitating Factors: probable trigg...,worksheet,5Ps-case-formulation,3,2
3,Identify Perpetuating Factors: what keeps the ...,worksheet,5Ps-case-formulation,4,3
4,Identify Protective Factors: what are the posi...,worksheet,5Ps-case-formulation,5,4
5,Describe the Situation / Trigger that led to t...,worksheet,7-column-thought-record,1,5
6,Identify your Feelings / Emotions and rate the...,worksheet,7-column-thought-record,2,6
7,Notice your Body Sensations related to the emo...,worksheet,7-column-thought-record,3,7
8,Write down Unhelpful Thoughts / Images that ca...,worksheet,7-column-thought-record,4,8
9,Record Facts that support the unhelpful thought.,worksheet,7-column-thought-record,5,9


In [ ]:
# Make sure worksheet text is string type
worksheet_df['text'] = worksheet_df['text'].astype(str)

In [ ]:
user_chunks = []

for idx, row in df.iterrows():
    input_statements = row['input']  # list of simple sentences
    response_text = row['output']

    for stmt in input_statements:
        user_chunks.append({
            "text": stmt,
            "source": "mental_chat",
            "type": "user_expression",
            "response": response_text
        })

import pandas as pd
user_chunks_df = pd.DataFrame(user_chunks)
user_chunks_df['chunk_id'] = range(len(user_chunks_df))

In [ ]:
user_chunks_df

,text,source,type,response,chunk_id
0,I have been struggling with my mental health f...,mental_chat,user_expression,I understand that you have been dealing with a...,0
1,I can not seem to find a way to cope with it .,mental_chat,user_expression,I understand that you have been dealing with a...,1
2,"I have tried visualization , positive thinking ,",mental_chat,user_expression,I understand that you have been dealing with a...,2
3,"even medication ,",mental_chat,user_expression,I understand that you have been dealing with a...,3
4,nothing seems to work .,mental_chat,user_expression,I understand that you have been dealing with a...,4
...,...,...,...,...,...
192022,I really need this job .,mental_chat,user_expression,hmm this is a tough one!,192022
192023,"People keep telling me I have "" anxiety """,mental_chat,user_expression,hmm this is a tough one!,192023
192024,I am terrified of having an anxiety attack on ...,mental_chat,user_expression,hmm this is a tough one!,192024
192025,This is all new to me .,mental_chat,user_expression,hmm this is a tough one!,192025


In [ ]:
# For worksheet_df, make the columns consistent
worksheet_rag_df = worksheet_df.rename(columns={"text": "text"})

# Select only relevant columns for RAG
worksheet_rag_df = worksheet_rag_df[['text', 'source', 'technique', 'step', 'chunk_id']]

# For user_chunks_df, add missing columns to match structure
user_chunks_df['technique'] = "user-expression"
user_chunks_df['step'] = None  # no step
user_chunks_df = user_chunks_df[['text', 'source', 'technique', 'step', 'chunk_id', 'response']]

# Merge
rag_df = pd.concat([worksheet_rag_df, user_chunks_df], ignore_index=True, sort=False)
rag_df = rag_df.reset_index(drop=True)

In [ ]:
rag_df

,text,source,technique,step,chunk_id,response
0,List the Presenting Problems the client is fac...,worksheet,5Ps-case-formulation,1,0,NaN
1,Identify Predisposing Factors: what has happen...,worksheet,5Ps-case-formulation,2,1,NaN
2,Identify Precipitating Factors: probable trigg...,worksheet,5Ps-case-formulation,3,2,NaN
3,Identify Perpetuating Factors: what keeps the ...,worksheet,5Ps-case-formulation,4,3,NaN
4,Identify Protective Factors: what are the posi...,worksheet,5Ps-case-formulation,5,4,NaN
...,...,...,...,...,...,...
192035,I really need this job .,mental_chat,user-expression,None,192022,hmm this is a tough one!
192036,"People keep telling me I have "" anxiety """,mental_chat,user-expression,None,192023,hmm this is a tough one!
192037,I am terrified of having an anxiety attack on ...,mental_chat,user-expression,None,192024,hmm this is a tough one!
192038,This is all new to me .,mental_chat,user-expression,None,192025,hmm this is a tough one!


In [ ]:
rag_df.to_csv("rag_df.csv")

In [ ]:
rag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192040 entries, 0 to 192039
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       192040 non-null  object
 1   source     192040 non-null  object
 2   technique  192040 non-null  object
 3   step       13 non-null      object
 4   chunk_id   192040 non-null  int64 
 5   response   192027 non-null  object
dtypes: int64(1), object(5)
memory usage: 8.8+ MB
